In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, Input,InputLayer, Activation, BatchNormalization
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import cv2
import os
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
from keras.callbacks import EarlyStopping

In [2]:
labels = ['NORMAL', 'PNEUMONIA']
img_size = 224

def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img = cv2.imread(img_path)
                if img is None:
                    continue  # Skip if the image cannot be loaded

                resized_arr = cv2.resize(img, (img_size, img_size))
                
                contrast = int((177 - 0) * (127 - (-127)) / (254 - 0) + (-127))
                Alpha = float(131 * (contrast + 127)) / (127 * (131 - contrast))
                Gamma = 127 * (1 - Alpha)
                cntrst = cv2.addWeighted(resized_arr, Alpha, resized_arr, 0, Gamma)
                
                data.append([cntrst, class_num])
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
    return np.array(data, dtype=object)

train = get_data(r"D:\purpose.D drive\pneumoniaxray\xray_chest\train")
test = get_data(r"D:\purpose.D drive\pneumoniaxray\xray_chest\test")
val = get_data(r"D:\purpose.D drive\pneumoniaxray\xray_chest\val")

In [3]:
x_train = []
y_train = []
x_val=[]
y_val=[]
x_test = []
y_test = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in test:
  x_test.append(feature)
  y_test.append(label)

for feature, label in val:
  x_val.append(feature)
  y_val.append(label)

In [4]:
# Normalize the data
x_train = np.array(x_train,dtype=np.float16) / 255
x_test = np.array(x_test,dtype=np.float16) / 255
x_val = np.array(x_val,dtype=np.float16) / 255

In [5]:
x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

In [6]:
# Load the saved models
resnet50_model = tf.keras.models.load_model(r'D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\restnet50saved.h5')
inception_model = tf.keras.models.load_model(r'D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\inceptionv3saved.h5')
vgg19_model = tf.keras.models.load_model(r'D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\vgg19saved.h5')

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt

# Function to calculate and print metrics
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    return accuracy, precision, recall, f1


In [8]:
vgg19_predictions = vgg19_model.predict(x_val)
vgg19_predicted_classes = (vgg19_predictions > 0.5).astype(int).flatten()
print("VGG19 Model Metrics:")
calculate_metrics(y_val.flatten(), vgg19_predicted_classes)

# Predict and calculate metrics for ResNet50 model
resnet50_predictions = resnet50_model.predict(x_val)
resnet50_predicted_classes = (resnet50_predictions > 0.5).astype(int).flatten()
print("\nResNet50 Model Metrics:")
calculate_metrics(y_val.flatten(), resnet50_predicted_classes)

# Predict and calculate metrics for InceptionV3 model
inceptionv3_predictions = inception_model.predict(x_val)
inceptionv3_predicted_classes = (inceptionv3_predictions > 0.5).astype(int).flatten()
print("\nInceptionV3 Model Metrics:")
calculate_metrics(y_val.flatten(), inceptionv3_predicted_classes)

2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step
VGG19 Model Metrics:
Accuracy: 0.9444444444444444
Precision: 0.9
Recall: 1.0
F1 Score: 0.9473684210526315
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 5s/step

ResNet50 Model Metrics:
Accuracy: 0.9722222222222222
Precision: 0.9473684210526315
Recall: 1.0
F1 Score: 0.972972972972973


1/2 ━━━━━━━━━━━━━━━━━━━━ 6s 7s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000018B45CD5BD0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step

InceptionV3 Model Metrics:
Accuracy: 0.8611111111111112
Precision: 0.782608695652174
Recall: 1.0
F1 Score: 0.8780487804878049


(0.8611111111111112, 0.782608695652174, 1.0, 0.8780487804878049)

CLASSIFICATION REPORT OF VGG19

In [9]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming you have your test data (x_test and y_test)
# Make predictions for the test set
predictions = vgg19_model.predict(x_test)

# Convert the predicted probabilities to class labels for binary classification
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Ensure y_test is in binary labels format (0 or 1)
y_test_classes = y_test.flatten()  # Flatten if necessary, assuming y_test is already shaped appropriately

# Print the classification report
print(classification_report(y_test_classes, predicted_classes, target_names=['NORMAL (Class 0)', 'PNEUMONIA (Class 1)']))

# Calculate and print the accuracy
accuracy = accuracy_score(y_test_classes, predicted_classes)
print(f"Accuracy: {accuracy:.4f}")

# Calculate and print the precision and recall
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test_classes, predicted_classes, zero_division=1)
recall = recall_score(y_test_classes, predicted_classes, zero_division=1)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 260s 13s/step
                     precision    recall  f1-score   support

   NORMAL (Class 0)       0.98      0.44      0.61       234
PNEUMONIA (Class 1)       0.75      0.99      0.85       390

           accuracy                           0.79       624
          macro avg       0.87      0.72      0.73       624
       weighted avg       0.84      0.79      0.76       624

Accuracy: 0.7885
Precision: 0.7490
Recall: 0.9949


CLASSIFICATION REPORT OF INCEPTIONV3

In [10]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming you have your test data (x_test and y_test)
# Make predictions for the test set
predictions = inception_model.predict(x_test)

# Convert the predicted probabilities to class labels for binary classification
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Ensure y_test is in binary labels format (0 or 1)
y_test_classes = y_test.flatten()  # Flatten if necessary, assuming y_test is already shaped appropriately

# Print the classification report
print(classification_report(y_test_classes, predicted_classes, target_names=['NORMAL (Class 0)', 'PNEUMONIA (Class 1)']))

# Calculate and print the accuracy
accuracy = accuracy_score(y_test_classes, predicted_classes)
print(f"Accuracy: {accuracy:.4f}")

# Calculate and print the precision and recall
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test_classes, predicted_classes, zero_division=1)
recall = recall_score(y_test_classes, predicted_classes, zero_division=1)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step
                     precision    recall  f1-score   support

   NORMAL (Class 0)       0.98      0.48      0.65       234
PNEUMONIA (Class 1)       0.76      0.99      0.86       390

           accuracy                           0.80       624
          macro avg       0.87      0.74      0.76       624
       weighted avg       0.84      0.80      0.78       624

Accuracy: 0.8029
Precision: 0.7623
Recall: 0.9949


CLASSIFICATION REPORT OF RESTNET50

In [11]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming you have your test data (x_test and y_test)
# Make predictions for the test set
predictions = resnet50_model.predict(x_test)

# Convert the predicted probabilities to class labels for binary classification
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Ensure y_test is in binary labels format (0 or 1)
y_test_classes = y_test.flatten()  # Flatten if necessary, assuming y_test is already shaped appropriately

# Print the classification report
print(classification_report(y_test_classes, predicted_classes, target_names=['NORMAL (Class 0)', 'PNEUMONIA (Class 1)']))

# Calculate and print the accuracy
accuracy = accuracy_score(y_test_classes, predicted_classes)
print(f"Accuracy: {accuracy:.4f}")

# Calculate and print the precision and recall
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test_classes, predicted_classes, zero_division=1)
recall = recall_score(y_test_classes, predicted_classes, zero_division=1)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step
                     precision    recall  f1-score   support

   NORMAL (Class 0)       1.00      0.43      0.60       234
PNEUMONIA (Class 1)       0.75      1.00      0.85       390

           accuracy                           0.79       624
          macro avg       0.87      0.72      0.73       624
       weighted avg       0.84      0.79      0.76       624

Accuracy: 0.7869
Precision: 0.7457
Recall: 1.0000


In [12]:
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Function to load and evaluate a model
def evaluate_model(model_path, x_val, y_val):
    # Load the saved model
    model = tf.keras.models.load_model(model_path)

    # Make predictions for the validation set
    predictions = model.predict(x_val)
    
    # Convert the predicted probabilities to class labels for binary classification
    predicted_classes = (predictions > 0.5).astype(int).flatten()

    # Ensure y_val is in binary labels format (0 or 1)
    y_val_classes = y_val.flatten()

    # Calculate accuracy, precision, recall, and F1 score
    accuracy = accuracy_score(y_val_classes, predicted_classes)
    precision = precision_score(y_val_classes, predicted_classes, zero_division=1)
    recall = recall_score(y_val_classes, predicted_classes, zero_division=1)
    f1 = f1_score(y_val_classes, predicted_classes, zero_division=1)
    
    # Return the calculated metrics
    return accuracy, precision, recall, f1


In [13]:
# List of model paths
model_paths = [
    r'D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\restnet50saved.h5',
    r'D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\inceptionv3saved.h5',
    r'D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\vgg19saved.h5'
]

# Dictionary to store the results
results = {}

# Evaluate each model and store the results
for model_path in model_paths:
    accuracy, precision, recall, f1 = evaluate_model(model_path, x_val, y_val)
    results[model_path] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step


2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step


2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step


In [14]:
# Print the results for each model
for model_path, metrics in results.items():
    print(f"Model: {model_path}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1_score']:.4f}")
    print("\n")

Model: D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\restnet50saved.h5
Accuracy: 0.9722
Precision: 0.9474
Recall: 1.0000
F1 Score: 0.9730


Model: D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\inceptionv3saved.h5
Accuracy: 0.8611
Precision: 0.7826
Recall: 1.0000
F1 Score: 0.8780


Model: D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\vgg19saved.h5
Accuracy: 0.9444
Precision: 0.9000
Recall: 1.0000
F1 Score: 0.9474




In [15]:

# Find the best model based on accuracy
best_model = max(results, key=lambda x: results[x]['accuracy'])
print(f"Best Model: {best_model}")
print(f"Best Model Accuracy: {results[best_model]['accuracy']:.4f}")
print(f"Best Model Precision: {results[best_model]['precision']:.4f}")
print(f"Best Model Recall: {results[best_model]['recall']:.4f}")
print(f"Best Model F1 Score: {results[best_model]['f1_score']:.4f}")

Best Model: D:\purpose.D drive\pneumoniaxray\xray_chest\final upload drive\restnet50saved.h5
Best Model Accuracy: 0.9722
Best Model Precision: 0.9474
Best Model Recall: 1.0000
Best Model F1 Score: 0.9730
